In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import nidaqmx

In [ ]:
from nidaqmx.stream_readers import AnalogMultiChannelReader, AnalogSingleChannelReader
from nidaqmx.stream_writers import AnalogMultiChannelWriter, AnalogSingleChannelWriter

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from nidaqmx.constants import Edge, AcquisitionType

In [ ]:
from numba import jit

In [ ]:
sample_rate = 500000.0
duration = 1
n_samples = int(round(duration*sample_rate))
sine_frequency = 200
time = np.linspace(0, duration, n_samples)
signal = np.sin(time*sine_frequency*2*np.pi)*3
double_signal = np.stack([signal, signal], 0)
with nidaqmx.Task() as write_task, nidaqmx.Task() as read_task:
    # Configure the channels
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1:2", min_val=-5, max_val=5)
    write_task.ao_channels.add_ao_voltage_chan("Dev1/ao0:1", min_val=-5, max_val=5)
    
    # Set the timing of both to the onboard clock so that they are synchronised
    read_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="OnboardClock",
                                         active_edge = Edge.RISING,
                                         sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples)
    write_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="OnboardClock",
                                          active_edge = Edge.RISING,
                                          sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples)
    
    # This is necessary to synchronise reading and wrting
    read_task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/ao/StartTrigger", Edge.RISING)
    
    writer = AnalogMultiChannelWriter(write_task.out_stream)
    reader = AnalogMultiChannelReader(read_task.in_stream)
    for i in tqdm(range(10)):
        # The first write has to be defined before the task starts
        writer.write_many_sample(double_signal)
        if i == 0:
            read_task.start()
            write_task.start()

        values_read = np.zeros((2,n_samples), dtype=np.float64)
        reader.read_many_sample(
            values_read, number_of_samples_per_channel=n_samples,
            timeout=11)

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
for i in range(2):
    axes[i].plot(time, double_signal[i,:])
    axes[i].plot(time, values_read[i,:])